In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

import glob
import json
import re


from nltk.stem import WordNetLemmatizer
from nltk.collocations import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))"""

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Similarity compute
import gensim

In [3]:
root_path = 'E:\Machathon1.1'

#all_files = glob.glob(f'{root_path}/documnet_parses/**/*.json', recursive=True)
files_path = []
for dirname, _, filenames in os.walk(f'{root_path}\pmc_json'):
    for filename in filenames:
        files_path.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk(f'{root_path}\pdf_json'):
    for filename in filenames:
        files_path.append(os.path.join(dirname, filename))

In [4]:
#TODO: when you finish, remove that list concatenation for running on all the data set
files_path = files_path[0:10000]

In [5]:
def remove_html(text):
    return re.sub('<.*?>','',text)

def remove_URL(text):
    return re.sub('https?:\/\/[^\s]+', '', text)

def lemmatization(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    return " ".join([wordnet_lemmatizer.lemmatize(t) for t in text.split()]) 

def stemming(text):
    newtext =''
    lancaster=LancasterStemmer()
    for i in text.split():
        newtext += ' ' + lancaster.stem(i)
    return newtext

def remove_punc(text):
    puncs = '!"#$%&()*+,./:;<=>?@[\\]^_`{|}~'
    for punc in puncs:
        text = ' '.join(text.split(punc))
    return text
def remove_stopWords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

In [6]:
def preprocess(text):
    text = remove_html(text)
    text = remove_URL(text)
    text = lemmatization(text)
    text = remove_punc(text)
    text = remove_stopWords(text)
    return text.lower().strip()

In [9]:
#loading the files
files = []
with open('articles_body_.csv', encoding='latin-1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        files.append(row)

In [10]:
len(files)

9262

In [16]:
files[0:2]

[['middle east respiratory syndrome coronavirus mers-cov discovered 2012 cause fatal respiratory disease humans although mers-cov might originated bats dromedary camel camelus dromedarius natural host likely source human mers-cov infection 1 2 camel trade major driver mers-cov movement africa arabian peninsula 3 human case occurred rajastan india large breeding center dromedaries exported pakistan bangladesh seropositive dromedary identified pakistan little known mers-cov part south asia 4 in bangladesh camel bred farm imported india sale seasonal market ritual slaughter religious festivals imported camel go directly urban market sold trader separate enterprise farmed camels during septemberâ\x80\x93october 2015 festival eid-ul-adha collected tested coronaviruses specimen 36 dromedary camel urban farm 19 camel 18 fat-tailed sheep urban market capital city dhaka table technical appendix the testing wa conducted part us agency international developmentâ\x80\x99s predict program conduct s

In [14]:
def tagged_files(indx, file):
    tokens = word_tokenize(file)
    return gensim.models.doc2vec.TaggedDocument(tokens, [indx])

In [27]:
files = [' '.join(str(c) for c in file) for file in files]

In [28]:
files[0:2]

['middle east respiratory syndrome coronavirus mers-cov discovered 2012 cause fatal respiratory disease humans although mers-cov might originated bats dromedary camel camelus dromedarius natural host likely source human mers-cov infection 1 2 camel trade major driver mers-cov movement africa arabian peninsula 3 human case occurred rajastan india large breeding center dromedaries exported pakistan bangladesh seropositive dromedary identified pakistan little known mers-cov part south asia 4 in bangladesh camel bred farm imported india sale seasonal market ritual slaughter religious festivals imported camel go directly urban market sold trader separate enterprise farmed camels during septemberâ\x80\x93october 2015 festival eid-ul-adha collected tested coronaviruses specimen 36 dromedary camel urban farm 19 camel 18 fat-tailed sheep urban market capital city dhaka table technical appendix the testing wa conducted part us agency international developmentâ\x80\x99s predict program conduct su

In [29]:
Files_tokens = [tagged_files(indx, file) for indx, file in enumerate(files)]

In [30]:
Files_tokens[:2]

[TaggedDocument(words=['middle', 'east', 'respiratory', 'syndrome', 'coronavirus', 'mers-cov', 'discovered', '2012', 'cause', 'fatal', 'respiratory', 'disease', 'humans', 'although', 'mers-cov', 'might', 'originated', 'bats', 'dromedary', 'camel', 'camelus', 'dromedarius', 'natural', 'host', 'likely', 'source', 'human', 'mers-cov', 'infection', '1', '2', 'camel', 'trade', 'major', 'driver', 'mers-cov', 'movement', 'africa', 'arabian', 'peninsula', '3', 'human', 'case', 'occurred', 'rajastan', 'india', 'large', 'breeding', 'center', 'dromedaries', 'exported', 'pakistan', 'bangladesh', 'seropositive', 'dromedary', 'identified', 'pakistan', 'little', 'known', 'mers-cov', 'part', 'south', 'asia', '4', 'in', 'bangladesh', 'camel', 'bred', 'farm', 'imported', 'india', 'sale', 'seasonal', 'market', 'ritual', 'slaughter', 'religious', 'festivals', 'imported', 'camel', 'go', 'directly', 'urban', 'market', 'sold', 'trader', 'separate', 'enterprise', 'farmed', 'camels', 'during', 'septemberâ\x80\

In [31]:
#TODO: Hyperparameters selection
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2)
model.build_vocab(Files_tokens)
model.train(Files_tokens, total_examples=model.corpus_count, epochs=model.iter)

70975049

In [32]:
#questions on task
questions = ["What do we know about potential risks factors?",
             "what is the effect of Smoking, pre-existing pulmonary disease?",
             "Do co-existing respiratory/viral infections make the virus more transmissible or virulent and other comorbidities?",
             "What is the effect on Neonates and pregnant women?",
             "What are the Socio-economic and behavioral factors on COVID-19?",
             "What is the economic impact of the virus?",
             "What are Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors?",
             "Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups",
             "What are the Susceptibility of populations?",
             "What are the Public health mitigation measures that could be effective for control?"]

In [33]:
queries_token = [model.infer_vector(word_tokenize(q)) for q in questions]
queries_token[:2]

[array([ 0.09454162, -0.17507713,  0.14527556, -0.07600038,  0.06668272,
        -0.19183695, -0.0542518 , -0.10649228,  0.11414116,  0.05361579,
         0.0367853 , -0.0898709 , -0.10507233, -0.136382  , -0.00359952,
         0.06653671,  0.07869028, -0.08203386, -0.00601415, -0.12010124,
         0.12828426,  0.06961537, -0.12268525,  0.0613179 , -0.01736771,
        -0.18182708,  0.02827813, -0.04904317,  0.09296063,  0.01076553,
        -0.06122062, -0.09138717, -0.0028312 , -0.0211558 ,  0.01914454,
         0.14210613,  0.06830002, -0.03465704,  0.02177076, -0.06127782,
         0.0364909 ,  0.22535396, -0.06568258,  0.01647367,  0.03909424,
        -0.05605043,  0.02787719, -0.03217257, -0.03468897, -0.04943793],
       dtype=float32),
 array([-0.00833405, -0.02929661,  0.09792207,  0.0431366 ,  0.21422498,
        -0.00757215, -0.05564119, -0.03703225,  0.00722412,  0.00781121,
         0.0416716 , -0.06399949, -0.03920009,  0.0368845 , -0.01445653,
         0.0345287 ,  0.056

In [34]:
similar_docs = [model.docvecs.most_similar([query],topn=20) for query in queries_token]
similar_docs[:2]
#Question: how to get the 8098 article, I guess it refers to articles!!!

[[(589, 0.7527582049369812),
  (1181, 0.7331005930900574),
  (778, 0.7294313907623291),
  (2941, 0.728471040725708),
  (3101, 0.7240861654281616),
  (3993, 0.718614935874939),
  (8750, 0.7122551202774048),
  (7258, 0.7040449380874634),
  (795, 0.704017162322998),
  (5882, 0.7039496302604675),
  (9050, 0.7033720016479492),
  (3048, 0.7026360034942627),
  (3391, 0.7012667655944824),
  (6611, 0.6995251178741455),
  (4958, 0.6978008151054382),
  (2195, 0.6976174712181091),
  (310, 0.6965381503105164),
  (2748, 0.694340705871582),
  (7736, 0.6939170956611633),
  (6593, 0.6928354501724243)],
 [(7869, 0.5793647766113281),
  (2156, 0.5661539435386658),
  (5260, 0.560154914855957),
  (5431, 0.5568708777427673),
  (1269, 0.5517193078994751),
  (7089, 0.5488781929016113),
  (2835, 0.5432465672492981),
  (2849, 0.53328537940979),
  (3233, 0.5321711897850037),
  (4426, 0.5270989537239075),
  (4657, 0.5253636837005615),
  (3376, 0.5227980613708496),
  (6897, 0.5092020034790039),
  (615, 0.5083332061

In [35]:
type(similar_docs)

list

In [37]:
documents_found = [[files[doc[0]] for doc in question] for question in similar_docs]
documents_found

[['the step outlined checklist developed based consensus expert opinion currently employed institution we using ongoing patient tracking contact program inform utility checklist although checklist ha formally studied respect impact outcomes believe serve blueprint institution tackling spread covid-19',
  'the condition patient severe sars-covâ\x80\x912 infection patient might deteriorate rapidly acute respiratory distress syndrome septic shock eventually followed multiple organ failure fulminant myocarditis more attention paid patient extremely increased cardiac troponin i ctni level new-onset arrhythmias the application mechanical respirator circulatory support systems including iabp impella ecmo might beneficial effect patients',
  'nscriptional translational level moreover virus-induced rrna decrease wa described flaviviral infection first time the research contribute understanding mechanism determine least extent subsequent pathological processes however relatively late onset effec

In [38]:
type(documents_found)

list

In [39]:
import joblib
# Save the model to disk
joblib.dump(model, 'nlp_model.pkl')

['nlp_model.pkl']